Update survey information:
- Survey name
- Survey description and 
- Observers


## Set-up
Load libraries 

In [1]:
import openpyxl
from pathlib import Path
import os
from datetime import datetime
from configparser import ConfigParser
import psycopg2
from psycopg2.extensions import AsIs
import pyprojroot

Load functions from `lib` folder, we will use a function to read db credentials and one for batch insert and updates:

In [3]:
from lib.parseparams import read_dbparams

Define path to workbooks

In [4]:
repodir = pyprojroot.find_root(pyprojroot.has_dir(".git"))
inputdir = repodir / "data" / "input-field-form"

Database credentials are stored in a database.ini file

In [5]:
dbparams = read_dbparams(repodir / 'secrets' / 'database.ini', section='aws-lght-sl')

In [21]:
print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(**dbparams)
cur = conn.cursor()

qrystr = """UPDATE form.surveys SET survey_description=%s WHERE survey_name=%s;"""

qry = cur.mogrify(qrystr, ('Newnes plateau swamps','NEWNES'))
cur.execute(qry)
qry = cur.mogrify(qrystr, ('Kosciuszko NP Alpine Ash','KNP AlpAsh'))
cur.execute(qry)
qry = cur.mogrify(qrystr, ('Southern NSW Rainforests','SthnNSWRF'))
cur.execute(qry)


cur.execute("UPDATE form.surveys SET survey_description=survey_name WHERE survey_description is NULL;")

cur.execute("SELECT survey_name,survey_description, observers FROM form.surveys;")
surveyinfo = cur.fetchall()


cur.execute("SELECT userkey,givennames,surname FROM form.observerid;")
observerid = cur.fetchall()

cur.close()
conn.commit()        
if conn is not None:
    conn.close()
    print('Database connection closed.')

Connecting to the PostgreSQL database...
Database connection closed.


In [22]:
observerid

[(7, 'David', 'Keith'),
 (9, 'D.', 'Benson'),
 (10, 'L.', 'Watts,'),
 (11, 'T.', 'Manson'),
 (12, 'Jackie', 'Miles'),
 (13, 'Robert', 'Kooyman'),
 (8, 'Alexandria', 'Thomsen'),
 (14, 'Jedda', 'Lemmen')]

In [23]:
surveyinfo

[('TO BE CLASSIFIED',
  'Placeholder for field visits not yet assigned to a survey',
  'JR Ferrer-Paris'),
 ('UplandBasalt', 'Upland Basalt', None),
 ('Rainforests NSW-Qld', 'Rainforests NE NSW & SE Qld', None),
 ('NEWNES', 'Newnes plateau swamps', None),
 ('KNP AlpAsh', 'Kosciuszko NP Alpine Ash', None),
 ('SthnNSWRF', 'Southern NSW Rainforests', None),
 ('Alpine Bogs', 'Alpine Bogs', None),
 ('Robertson RF', 'Robertson RF', None),
 ('Yatteyattah', 'Yatteyattah', None),
 ('Mallee Woodlands', 'Mallee Woodlands', None)]